## Tarea

Implementar un sistema automático reconocedor de caracteres trazados con el mouse, los caracteres son os dígitos del 0 al 9. Por cada dígito, entrenar un modelo oculto de Markov discreto del tipo izquierda-derecha utilizando varios ejemplos de entrenamiento por cada modelo (Ej 3). El clasificador deberá decidir por el carácter asociado al modelo que tenga la mayor probabilidad de haber producido la secuencia de observaciones (secuencia de pendientes cuantificadas) del carácter que se pretende reconocer. Utilizar los algoritmos que vimos en clase (Algoritmo Forward para clasificar y algoritmo Baum-Welch para entrenar), esto quiere decir, no utilizar una librería de modelos ocultos de Markov (A menos que sea HTK o que no usen modelos discretos sino semi-contínuos de Markov). El programa debe leer los movimientos del mouse y reportar el carácter reconocido en cuanto se termine el trazo de dicho carácter.



In [17]:
def restarle(arr):
    newarr = []
    for i in arr:
        newarr.append(i-1)
    return newarr

In [18]:

# Obs array to train models 
# 0
dig0 = [8, 8, 6, 5, 5, 4, 4, 3, 2, 1, 0, 0, 15, 14, 13, 12, 12, 11, 10, 9, 9]
dig00 = [8, 7, 7, 5, 4, 3, 1, 0, 14, 14, 13, 12, 12, 11]
dig000 = [8, 7, 6, 5, 4, 4, 3, 3, 2, 1, 0, 0, 0, 13, 11, 11, 11, 10, 10, 9]
# 1
dig1 = [4, 4, 4, 4, 4, 4, 4, 4]
dig01 = [4, 4, 4, 4, 4, 4, 3]
dig001 = [4, 4, 4, 4, 4, 4, 4, 4, 5]
# 2
dig2 = [14, 14, 15, 2, 5, 6, 6, 5, 5, 2, 0, 0, 0]
dig02 = [13, 14, 15, 0, 2, 4, 5, 6, 6, 6, 6, 6, 5, 1, 0, 0, 0, 0]
dig002 = [13, 14, 0, 2, 3, 4, 5, 6, 6, 6, 6, 1, 1, 1]
# 3
dig3 = [0, 0, 0, 4, 6, 7, 7, 0, 0, 2, 4, 6, 7, 8]
dig03 = [0, 0, 0, 0, 7, 7, 7, 7, 1, 0, 1, 2, 5, 6, 7, 8, 8]
dig003 = [0, 0, 1, 4, 6, 7, 8, 0, 0, 0, 1, 3, 5, 6, 7, 8, 8]
# 4
dig4 = [8, 8, 8, 8, 8, 8, 15, 15, 15, 14, 14, 14, 14, 14, 5, 4, 4, 4, 4, 4, 4, 4]
dig04 = [8, 8, 8, 8, 8, 8, 15, 15, 15, 14, 14, 14, 14, 2, 4, 4, 4, 4, 4]
dig004 = [7, 8, 8, 8, 8, 15, 0, 15, 13, 13, 13, 13, 1, 4, 4, 4, 4, 4]
# 5
dig5 = [8, 8, 8, 8, 8, 7, 3, 4, 0, 0, 0, 1, 2, 2, 3, 4, 6, 7, 8, 9, 9]
dig05 = [8, 8, 8, 7, 4, 3, 0, 0, 0, 2, 2, 3, 4, 6, 7, 8, 8]
dig005 = [8, 8, 8, 7, 4, 4, 0, 0, 0, 1, 1, 3, 4, 5, 7, 8, 9, 10, 11]
# 6
dig6 = [7, 6, 6, 5, 4, 3, 1, 0, 15, 13, 12, 9, 7, 7, 6]
dig06 = [8, 7, 6, 5, 4, 3, 3, 2, 1, 1, 0, 14, 12, 11, 9, 8, 7, 6, 5]
dig006 = [7, 7, 6, 5, 4, 4, 3, 2, 1, 0, 14, 12, 9, 7]
# 7
dig7 = [0, 0, 0, 0, 0, 0, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6]
dig07 = [0, 0, 0, 0, 0, 0, 2, 5, 5, 5, 5, 4, 5, 5, 5, 5]
dig007 = [0, 0, 0, 0, 0, 2, 6, 6, 6, 5, 5, 5, 5, 5, 5, 5]
# 8
dig8 = [8, 7, 4, 1, 0, 1, 2, 3, 5, 8, 9, 10, 11, 12, 14, 14, 14, 10]
dig08 = [10, 8, 7, 6, 4, 3, 0, 0, 0, 1, 2, 3, 5,
         6, 8, 8, 9, 9, 11, 12, 13, 14, 15, 15, 14, 13]
dig008 = [9, 8, 8, 5, 4, 3, 1, 1, 1, 1, 2, 3, 3, 4, 6, 8,
          8, 9, 10, 11, 12, 13, 14, 15, 15, 15, 13, 12, 10, 9]
# 9
dig9 = [9, 7, 5, 4, 1, 0, 12, 12, 4, 4, 4, 4, 4, 4]
dig09 = [8, 7, 5, 4, 2, 0, 14, 12, 12, 4, 4, 4, 4, 4, 4, 4]
dig009 = [8, 8, 6, 3, 1, 15, 12, 13, 4, 4, 3, 3, 4, 4, 4]


In [19]:
# Utils
import numpy as np


def stochastic(matrix):
    """Recives a matrix and returns it row stochastic, row sum equals 1"""
    return matrix/matrix.sum(axis=1)[:, None]


def smoothMatx(mat, smoothing=1e-5):
    """ sets floor value to 1e-5, smoothing value can be provided"""
    
    mat[mat == 0] = smoothing
    return mat


In [20]:
def logForward(V, a, b, pi):
    """Forward returns logprob"""
    T = V.shape[0]
    scale = np.zeros(T)
    alpha = np.zeros((T, a.shape[0]))

    alpha[0, :] = pi * b[:, V[0]]
    scale[0] = alpha[0, :].sum()
    # alpha scaled
    alpha[0] = alpha[0]/scale[0]
    # Induction steps
    for t in range(1, T):
        alpha_hat = alpha[t - 1].dot(a) * b[:, V[t]]
        # alpha scaling 
        scale[t] = alpha_hat.sum()
        alpha[t] = alpha_hat / scale[t]
    # print(alpha)
    return np.log(scale).sum()

def probforward(V, a, b, pi):
    """Forward returns prob, it may ocour underflow"""
    p = 1
    alpha = np.zeros((V.shape[0], a.shape[0]))
    alpha[0, :] = pi * b[:, V[0]]

    for t in range(1, V.shape[0]):
        probability_of_observation = 0  
        for j in range(a.shape[0]):
            alpha[t, j] = alpha[t - 1].dot(a[:, j]) * b[j, V[t]]
            probability_of_observation += alpha[t, j]  
        p = p * probability_of_observation  
    # print(alpha)
    return p  


def forward(V, a, b, pi):
    T = V.shape[0]
    alpha = np.zeros((T, a.shape[0]))
    alpha[0, :] = pi * b[:, V[0]]

    for t in range(1, T):
        for j in range(a.shape[0]):
            # Matrix Computation Steps
            alpha[t, j] = alpha[t - 1].dot(a[:, j]) * b[j, V[t]]

    return alpha


def backward(V, a, b):
    T = V.shape[0]
    beta = np.zeros((T, a.shape[0]))
    beta[T - 1] = np.ones((a.shape[0]))

    # Loop in backward way from T-1 to
    # Due to python indexing the actual loop will be T-2 to 0
    for t in range(T - 2, -1, -1):
        for j in range(a.shape[0]):
            beta[t, j] = (beta[t + 1] * b[:, V[t + 1]]).dot(a[j, :])

    return beta


def baum_welch(V, a, b, pi, n_iter=100):
    T = V.shape[0]
    M = a.shape[0]

    for n in range(n_iter):
        alpha = forward(V, a, b, pi)

        # print(alpha)
        beta = backward(V, a, b)

        # xi array m*m*t
        xi = np.zeros((M, M, T - 1))
        for t in range(T - 1):
            denominator = np.dot(
                np.dot(alpha[t, :].T, a) * b[:, V[t + 1]].T, beta[t + 1, :])
            for i in range(M):
                numerator = alpha[t, i] * a[i, :] * \
                    b[:, V[t + 1]].T * beta[t + 1, :].T
                xi[i, :, t] = numerator / denominator

        gamma = np.sum(xi, axis=1)
        # print(gamma)
        a = np.sum(xi, 2) / np.sum(gamma, axis=1).reshape((-1, 1))

        # Add additional T element in gamma
        gamma = np.hstack(
            (gamma, np.sum(xi[:, :, T - 2], axis=0).reshape((-1, 1))))

        K = b.shape[1]
        denominator = np.sum(gamma, axis=1)
        for l in range(K):
            b[:, l] = np.sum(gamma[:, V == l], axis=1)

        b = np.divide(b, denominator.reshape((-1, 1)))

    return {"a": a, "b": b}


In [21]:
# Recreamos el experimento de la presentación de Dr. Sung-Jung Cho de HMM

# Preflight Test

# obs 0 demo
V = np.array([7, 7, 6, 5, 5, 4, 4, 3, 1, 0, 15, 15, 14, 13, 12, 12, 11, 10, 9])
# V = np.array([8, 8, 6, 5, 5, 4, 4, 3, 2, 1, 0, 1,
#              15, 14, 13, 12, 12, 11, 10, 9, 9])

# Transition Probabilities
a = np.random.rand(3, 3)
a[1, 0] = 0
a[2, 0] = 0
a[2, 1] = 0
a = stochastic(a)

# Emission Probabilities
b = np.random.rand(3, 16)
b = stochastic(b)
pi = np.array((1, 0, 0))

# train model with demo 0 samples
fit1 = baum_welch(V, a, b, pi, n_iter=100)
print(f'Trans Mat: \n {fit1["a"]}')
print(f'Obs Mat: \n {fit1["b"]}')


a = fit1["a"]
b = fit1["b"]

# B matrix smoothing, change 0's to 1e-5
bsmooth = smoothMatx(b) 

# Classification with demos of 0 and 1 

p0 = probforward(np.array([7, 6, 5, 4, 3, 3, 2, 1, 0, 15, 14,
               13, 12, 11, 10, 9, 9, 8]), a, bsmooth, pi)  
print(f"0 prob: {p0}")
p1 = probforward(np.array([4, 4, 4, 4, 4]), a, bsmooth, pi)  
print(f"1 prob: {p1}")


logp0 = logForward(np.array([7, 6, 5, 4, 3, 3, 2, 1, 0, 15, 14,
               13, 12, 11, 10, 9, 9, 8]), a, bsmooth, pi) 
print(f"0 Logprob: {logp0}")

logp1 = logForward(np.array([4, 4, 4, 4, 4]), a, bsmooth, pi)
print(f"1 Logprob: {logp1}")


Trans Mat: 
 [[0.66666667 0.33333333 0.        ]
 [0.         0.8        0.2       ]
 [0.         0.         1.        ]]
Obs Mat: 
 [[0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000 1.45860863e-021 3.33333333e-001 6.66666667e-001
  0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000]
 [0.00000000e+000 6.54001791e-221 0.00000000e+000 2.00000000e-001
  4.00000000e-001 4.00000000e-001 1.01137367e-012 0.00000000e+000
  0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000]
 [9.09090909e-002 9.09090909e-002 0.00000000e+000 2.33821411e-016
  0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000 9.09090909e-002 9.09090909e-002 9.09090909e-002
  1.81818182e-001 9.09090909e-002 9.09090909e-002 1.81818182e-001]]
0 prob: 8.58223646789625e-191
1 prob: 3.393999840671142e-27
0 Logprob: 

In [173]:

# <==== Data for 0 ====>

# Transition Probabilities
a = np.random.rand(3, 3)
a[1, 0] = 0
a[2, 0] = 0
a[2, 1] = 0
a = stochastic(a)

# Emission Probabilities
b = np.random.rand(3, 16)
b = stochastic(b)
pi0 = np.array([1, 0, 0])

# train model with number 0 samples
fit0 = baum_welch(np.array(dig0), a, b, pi0)
# fit00 = baum_welch(V, a, b, pi0)
# fit000 = baum_welch(np.array(dig000), a, b, pi0)


# # <==== Data for 1 ====>

# Transition Probabilities
a = np.random.rand(2, 2)
a[1, 0] = 0
a = stochastic(a)

# Emission Probabilities
b = np.random.rand(2, 16)
b = stochastic(b)
pi1 = np.array((1, 0))

# train model with number 1 samples
fit1 = baum_welch(np.array(dig001), a, b, pi1)
# fit01 = baum_welch(V, a, b, pi)
# fit001 = baum_welch(V, a, b, pi)

# <==== Data for 2 ====>

# Transition Probabilities
a = np.random.rand(3, 3)
a[1, 0] = 0
a[2, 0] = 0
a[2, 1] = 0
a = stochastic(a)

# Emission Probabilities
b = np.random.rand(3, 16)
b = stochastic(b)
pi2 = np.array((1, 0, 0))

# train model with number 2 samples
fit2 = baum_welch(np.array(dig02), a, b, pi2)


# <==== Data for 3 ====>

a = np.random.rand(4, 4)
a = np.triu(a)
a = stochastic(a)

b = np.random.rand(4, 16)
b = stochastic(b)
pi3 = np.array((1, 0, 0, 0))

fit3 = baum_welch(np.array(dig003), a, b, pi3)


# <==== Data for 4 ====>

a = np.random.rand(3, 3)
a = np.triu(a)
a = stochastic(a)

b = np.random.rand(3, 16)
b = stochastic(b)
pi4 = np.array((1, 0, 0))

fit4 = baum_welch(np.array(dig004), a, b, pi4)


# <==== Data for 5 ====>

a = np.random.rand(4, 4)
a = np.triu(a)
a = stochastic(a)

b = np.random.rand(4, 16)
b = stochastic(b)
pi5 = np.array((1, 0, 0, 0))

fit5 = baum_welch(np.array(dig005), a, b, pi5)


# <==== Data for 6 ====>

a = np.random.rand(4, 4)
a = np.triu(a)
a = stochastic(a)

b = np.random.rand(4, 16)
b = stochastic(b)
pi6 = np.array((1, 0, 0, 0))

fit6 = baum_welch(np.array(dig006), a, b, pi6)


# <==== Data for 7 ====>

a = np.random.rand(2, 2)
a = np.triu(a)
a = stochastic(a)

b = np.random.rand(2, 16)
b = stochastic(b)
pi7 = np.array((1, 0))

fit7 = baum_welch(np.array(dig007), a, b, pi7)


# <==== Data for 8 ====>

a = np.random.rand(4, 4)
a = np.triu(a)
a = stochastic(a)

b = np.random.rand(4, 16)
b = stochastic(b)
pi8 = np.array((1, 0,0,0))

fit8 = baum_welch(np.array(dig008), a, b, pi8)


# <==== Data for 9 ====>

a = np.random.rand(4, 4)
a = np.triu(a)
a = stochastic(a)

b = np.random.rand(4, 16)
b = stochastic(b)
pi9 = np.array((1, 0,0,0))

fit9 = baum_welch(np.array(dig009), a, b, pi9)


def eval(obs):

    logprob = []

    logp0 = evalNum(np.array(obs), fit0["a"], fit0["b"], pi0)
    # logp00 = evalNum(np.array(obs), fit00["a"], fit00["b"], pi0)
    # logp000 = evalNum(np.array(obs), fit000["a"], fit000["b"], pi0)
    print(f"0 Logprob: {logp0}")
    logprob.append(logp0)

    logp1 = evalNum(np.array(obs), fit1["a"], fit1["b"], pi1)
    print(f"1 Logprob:{logp1}")
    logprob.append(logp1)

    logp2 = evalNum(np.array(obs), fit2["a"], fit2["b"], pi2)
    print(f"2 Logprob:{logp2}")
    logprob.append(logp2)

    logp3 = evalNum(np.array(obs), fit3["a"], fit3["b"], pi3)
    print(f"3 Logprob:{logp3}")
    logprob.append(logp3)

    logp4 = evalNum(np.array(obs), fit4["a"], fit4["b"], pi4)
    print(f"4 Logprob:{logp4}")
    logprob.append(logp4)

    logp5 = evalNum(np.array(obs), fit5["a"], fit5["b"], pi5)
    print(f"5 Logprob:{logp5}")
    logprob.append(logp5)

    logp6 = evalNum(np.array(obs), fit6["a"], fit6["b"], pi6)
    print(f"6 Logprob:{logp6}")
    logprob.append(logp6)

    logp7 = evalNum(np.array(obs), fit7["a"], fit7["b"], pi7)
    print(f"7 Logprob:{logp7}")
    logprob.append(logp7)

    logp8 = evalNum(np.array(obs), fit8["a"], fit8["b"], pi8)
    print(f"8 Logprob:{logp8}")
    logprob.append(logp8)

    logp9 = evalNum(np.array(obs), fit8["a"], fit9["b"], pi9)
    print(f"9 Logprob:{logp9}")
    logprob.append(logp9)

    maxlogprob = max(logprob)
    print(
        f"Number: {logprob.index(maxlogprob)} with logprob of:  {maxlogprob}")


def evalNum(obs, a, b, pi):
    b = smoothMatx(b)
    return logForward(np.array(obs), a, b, pi)


### Ventana que captura los trazos y emite las observaciones.

In [174]:
from tkinter import *
import math
import numpy as np
app = Tk()
app.geometry("400x400")


canvas = Canvas(app, bg='white')
canvas.pack(anchor='nw', fill='both', expand=1)

xcords = []
ycords = []


def get_x_and_y(event):
    global lasx, lasy
    lasx, lasy = event.x, event.y


def draw(event):
    global lasx, lasy
    xcords.append(lasx)
    ycords.append(lasy)
    canvas.create_line((lasx, lasy, event.x, event.y),
                       fill='red',
                       width=2)
    lasx, lasy = event.x, event.y


def release(event):
    print("Click release!")
    genObser(xcords, ycords)


def genObser(xlist, ylist):
    """Generate observation sequence"""
    xpurge, ypurge = [], []
    deglist = []
    obs = []
    for i in range(0, len(xlist), 20):
        xpurge.append(xlist[i])
        ypurge.append(ylist[i])

    for i in range(len(xpurge)-1):
        dy = ypurge[i]-ypurge[i+1]
        dx = xpurge[i]-xpurge[i+1]
        myradians = math.atan2(dy, dx)
        mydegrees = math.degrees(myradians)
        deglist.append(mydegrees)
        # print(mydegrees)

    for i in deglist:
        # 0
        if i >= 168.75 and i <= 180:
            obs.append(0)
        if i <= -168.75:
            obs.append(0)
        # 1
        if i > -168.75 and i < -146.25:
            obs.append(1)
        # 2
        if i > -146.25 and i < -123.75:
            obs.append(2)
        # 3
        if i > -123.75 and i < -101.25:
            obs.append(3)
        # 4
        if i > -101.25 and i < -78.75:
            obs.append(4)
        # 5
        if i > -78.75 and i < -56.25:
            obs.append(5)
        # 6
        if i > -56.25 and i < -33.75:
            obs.append(6)
        # 7
        if i > -33.75 and i < -11.25:
            obs.append(7)
        # 8
        if i >= 0 and i < 11.25:
            obs.append(8)
        if i < 0 and i > -11.25:
            obs.append(8)
        # 9
        if i >= 11.25 and i < 33.75:
            obs.append(9)
        # 10
        if i >= 33.75 and i < 56.25:
            obs.append(10)
        # 11
        if i >= 56.25 and i < 78.75:
            obs.append(11)
        # 12
        if i >= 78.75 and i < 101.25:
            obs.append(12)
        # 13
        if i >= 101.25 and i < 123.75:
            obs.append(13)
        # 14
        if i >= 123.75 and i < 146.25:
            obs.append(14)
        # 15
        if i >= 146.25 and i < 168.75:
            obs.append(15)
    print(f'Obs Sequence: {obs}')
    eval(np.array(obs))


canvas.bind("<Button-1>", get_x_and_y)
canvas.bind("<B1-Motion>", draw)
canvas.bind("<ButtonRelease-1>", release)

app.mainloop()


Click release!
Obs Sequence: [0, 0, 0, 0, 2, 6, 7, 6, 0, 0, 1, 4, 7]
0 Logprob: -112.5511038945658
1 Logprob:-138.38857722793028
2 Logprob:-82.48967543162689
3 Logprob:-48.154867888177144
4 Logprob:-84.98178397262721
5 Logprob:-91.89566718431584
6 Logprob:-86.05265337104116
7 Logprob:-76.52758979128096
8 Logprob:-107.4874423455913
9 Logprob:-116.5726800701647
Number: 3 with logprob of:  -48.154867888177144
